<a href="https://colab.research.google.com/github/kimjaehwankimjaehwan/psychology/blob/main/MBTI(Myers_Briggs_Type_Indicator).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


###Content

This dataset contains over 8600 rows of data, on each row is a person’s:

Type (This persons 4 letter MBTI code/type)
A section of each of the last 50 things they have posted (Each entry separated by "|||" (3 pipe characters))

###Acknowledgements

This data was collected through the PersonalityCafe forum, as it provides a large selection of people and their MBTI personality type, as well as what they have written.

In [2]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import matplotlib.pyplot as plt

file_path = '/content/drive/MyDrive/심리학/mbti_1.csv'
mbti_data = pd.read_csv(file_path)

def preprocess_text(text):
    text = text.lower()
    text = text.replace('|||', ' ')
    return text

mbti_data['processed_posts'] = mbti_data['posts'].apply(preprocess_text)

mbti_types = mbti_data['type'].unique()
type_word_freq = {}

for mbti_type in mbti_types:
    type_data = mbti_data[mbti_data['type'] == mbti_type]
    vectorizer = CountVectorizer(max_features=50, stop_words='english')  # 상위 50개 단어만 추출
    word_counts = vectorizer.fit_transform(type_data['processed_posts'])
    word_freq = word_counts.sum(axis=0)
    word_freq_dict = {word: word_freq[0, idx] for word, idx in vectorizer.vocabulary_.items()}
    type_word_freq[mbti_type] = word_freq_dict

for mbti_type, words in type_word_freq.items():
    print(f"MBTI 유형: {mbti_type}")
    print("상위 10개 단어:", sorted(words.items(), key=lambda x: x[1], reverse=True)[:10])
    print()


MBTI 유형: INFJ
상위 10개 단어: [('like', 11921), ('just', 9811), ('think', 8796), ('people', 8591), ('don', 8440), ('know', 6822), ('infj', 6472), ('ve', 6271), ('really', 6172), ('time', 4933)]

MBTI 유형: ENTP
상위 10개 단어: [('like', 5354), ('just', 4442), ('think', 3919), ('don', 3866), ('people', 3700), ('entp', 2995), ('know', 2782), ('really', 2590), ('ve', 2410), ('time', 1979)]

MBTI 유형: INTP
상위 10개 단어: [('like', 9732), ('just', 8098), ('don', 7761), ('think', 7098), ('people', 7096), ('know', 5043), ('intp', 4963), ('really', 4622), ('ve', 4425), ('com', 4170)]

MBTI 유형: INTJ
상위 10개 단어: [('like', 7793), ('don', 6320), ('just', 6248), ('people', 5950), ('think', 5557), ('intj', 4789), ('know', 4273), ('ve', 3748), ('really', 3472), ('time', 3347)]

MBTI 유형: ENTJ
상위 10개 단어: [('like', 1668), ('just', 1375), ('don', 1294), ('think', 1288), ('people', 1236), ('entj', 1019), ('know', 888), ('really', 731), ('ve', 725), ('time', 669)]

MBTI 유형: ENFJ
상위 10개 단어: [('like', 1652), ('just', 1304), (

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

tfidf_vectorizer = TfidfVectorizer(max_features=5000, stop_words='english')
X = tfidf_vectorizer.fit_transform(mbti_data['processed_posts'])

y = mbti_data['type']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LogisticRegression(max_iter=1000, multi_class='ovr')
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


Accuracy: 0.6530259365994237
              precision    recall  f1-score   support

        ENFJ       0.45      0.12      0.19        41
        ENFP       0.74      0.60      0.66       125
        ENTJ       0.79      0.34      0.48        44
        ENTP       0.72      0.58      0.64       135
        ESFJ       0.00      0.00      0.00         7
        ESFP       0.00      0.00      0.00         8
        ESTJ       0.00      0.00      0.00         7
        ESTP       1.00      0.13      0.24        15
        INFJ       0.65      0.70      0.68       288
        INFP       0.59      0.86      0.70       370
        INTJ       0.62      0.72      0.67       193
        INTP       0.69      0.81      0.74       293
        ISFJ       1.00      0.29      0.45        45
        ISFP       0.71      0.19      0.30        53
        ISTJ       0.67      0.23      0.34        44
        ISTP       0.75      0.45      0.56        67

    accuracy                           0.65      17

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


###**성능 분석**
#### 1.일부 유형에서의 낮은 성능:

ESFJ, ESFP, ESTJ와 같은 유형은 지원(support) 수가 매우 적고, 정밀도와 재현율이 0에 가까운 값을 보입니다. 이는 해당 유형의 데이터가 적어서 모델이 잘 학습하지 못했음을 의미합니다.

ISFJ와 ISTJ 유형도 비슷한 경향을 보이며, 이들 유형의 경우 재현율이 매우 낮습니다.

####2. 일부 유형에서의 상대적으로 높은 성능:

INFJ, INFP, INTJ, INTP 유형은 데이터가 많이 포함되어 있으며, F1 점수가 비교적 높습니다. 특히 INFP와 INTP는 재현율이 80%를 넘고, F1 점수도 0.70 이상으로 나옵니다.

이는 해당 유형의 데이터가 많아 모델이 잘 학습한 결과일 가능성이 큽니다.

####3. **매크로 평균(macro avg)**과 가중 평균(weighted avg):

매크로 평균은 모든 클래스의 평균 성능을 나타내며, 클래스의 데이터 수에 관계없이 동일하게 계산됩니다. 이 경우 성능이 좋지 않은 클래스들이 있어 평균적으로 낮은 값을 보입니다.

가중 평균은 각 클래스의 데이터 수에 비례하여 계산되므로, 데이터가 많은 클래스의 성능이 더 크게 반영됩니다. 따라서 매크로 평균보다 가중 평균이 더 높은 것을 볼 수 있습니다.

In [4]:
import numpy as np

tfidf_vectorizer = TfidfVectorizer(max_features=5000, stop_words='english')
X = tfidf_vectorizer.fit_transform(mbti_data['processed_posts'])
terms = np.array(tfidf_vectorizer.get_feature_names_out())

for mbti_type in mbti_types:
    type_data = mbti_data[mbti_data['type'] == mbti_type]
    type_tfidf = tfidf_vectorizer.transform(type_data['processed_posts'])
    mean_tfidf = np.mean(type_tfidf, axis=0).A1  # 각 단어의 평균 TF-IDF 값 계산
    top_terms_idx = mean_tfidf.argsort()[-10:][::-1]  # 상위 10개 단어 인덱스
    top_terms = terms[top_terms_idx]

    print(f"MBTI 유형: {mbti_type}")
    print("상위 10개 특징 단어:", top_terms)
    print()


MBTI 유형: INFJ
상위 10개 특징 단어: ['like' 'infj' 'just' 'think' 'people' 'don' 've' 'know' 'really' 'com']

MBTI 유형: ENTP
상위 10개 특징 단어: ['entp' 'like' 'just' 'don' 'think' 'people' 'know' 'really' 've' 'entps']

MBTI 유형: INTP
상위 10개 특징 단어: ['like' 'intp' 'just' 'don' 'people' 'think' 'com' 'know' 've' 'really']

MBTI 유형: INTJ
상위 10개 특징 단어: ['intj' 'like' 'don' 'just' 'people' 'think' 'know' 've' 'com' 'really']

MBTI 유형: ENTJ
상위 10개 특징 단어: ['entj' 'like' 'just' 'don' 'think' 'people' 'know' 'entjs' 've' 'type']

MBTI 유형: ENFJ
상위 10개 특징 단어: ['enfj' 'like' 'just' 'think' 'people' 'don' 'really' 'know' 've' 'enfjs']

MBTI 유형: INFP
상위 10개 특징 단어: ['like' 'infp' 'just' 'think' 'don' 'people' 'really' 've' 'com' 'know']

MBTI 유형: ENFP
상위 10개 특징 단어: ['enfp' 'like' 'just' 'think' 'don' 'people' 'really' 'know' 've' 'love']

MBTI 유형: ISFP
상위 10개 특징 단어: ['isfp' 'like' 'just' 'don' 'think' 'com' 'really' 'people' 'youtube'
 'www']

MBTI 유형: ISTP
상위 10개 특징 단어: ['istp' 'like' 'just' 'don' 'people' 'think'